# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows.
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [1]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:
            self.rows, self.cols = args
            self.matrix = [[0.0 for _ in range(self.cols)] for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.matrix = args[0]
            self.rows = len(self.matrix)
            self.cols = len(self.matrix[0]) if self.rows > 0 else 0
            for row in self.matrix:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns.")
        else:
            raise ValueError("Invalid constructor parameters.")

    def get_shape(self):
        return (self.rows, self.cols)

    def get_transpose(self):
        return Matrix([[self.matrix[j][i] for j in range(self.rows)] for i in range(self.cols)])

    def get_row(self, n):
        return Matrix([self.matrix[n]])

    def get_column(self, n):
        return Matrix([[self.matrix[i][n]] for i in range(self.rows)])

    def to_list(self):
        return self.matrix

    def get_block(self, row_start, row_end, col_start, col_end):
        return Matrix([row[col_start:col_end] for row in self.matrix[row_start:row_end]])

    def scalar_multiply(self, c):
        return Matrix([[c * self.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def add_matrix(self, N):
        if self.get_shape() != N.get_shape():
            raise ValueError("Matrices must be the same size to add.")
        return Matrix([[self.matrix[i][j] + N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def subtract_matrix(self, N):
        if self.get_shape() != N.get_shape():
            raise ValueError("Matrices must be the same size to subtract.")
        return Matrix([[self.matrix[i][j] - N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def multiply_matrix(self, N):
        if self.cols != N.rows:
            raise ValueError("Matrices dimensions must match for multiplication.")
        return Matrix([[sum(self.matrix[i][k] * N.matrix[k][j] for k in range(self.cols)) for j in range(N.cols)] for i in range(self.rows)])

    def element_wise_multiply(self, N):
        if self.get_shape() != N.get_shape():
            raise ValueError("Matrices must be the same size for element-wise multiplication.")
        return Matrix([[self.matrix[i][j] * N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def is_equal(self, N):
        return self.matrix == N.matrix

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.matrix[i][j]
        elif isinstance(index, slice):
            return Matrix([row[index] for row in self.matrix])
        elif isinstance(index, int):
            return Matrix([self.matrix[index]])
        else:
            raise ValueError("Invalid slicing parameters")

    def __setitem__(self, index, value):
        i, j = index
        self.matrix[i][j] = value

    def __eq__(self, other):
        return self.is_equal(other)

    def __add__(self, other):
        return self.add_matrix(other)

    def __sub__(self, other):
        return self.subtract_matrix(other)

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.multiply_matrix(other)
        elif isinstance(other, (int, float)):
            return self.scalar_multiply(other)
        else:
            raise ValueError("Unsupported multiplication type.")

    def __rmul__(self, other):
        return self.__mul__(other)

    def __str__(self):
        return "\n".join(["\t".join(map(str, row)) for row in self.matrix])


# Helper functions
def create_constant_matrix(n, m, c):
    return Matrix([[c] * m for _ in range(n)])

def create_zero_matrix(n, m):
    return Matrix([[0.0] * m for _ in range(n)])

def create_one_matrix(n, m):
    return Matrix([[1.0] * m for _ in range(n)])

def create_identity_matrix(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])


In [2]:
# Test the Matrix class

A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Test (AB)C = A(BC)
AB = A * B
BC = B * C
assert (AB * C).is_equal(A * BC), "(AB)C != A(BC)"

# Test 2 A(B + C) = AB + AC
B_plus_C = B + C
assert A * B_plus_C == A * B + A * C, "A(B+C) != AB + AC"

# Test 3 A * B != B * A
assert A * B != B * A, "AB == BA"

# Test 4 A * I = A
I = create_identity_matrix(2)
assert A * I == A, "A * I != A"


# Test block extraction (submatrix)
block = A.get_block(0, 2, 0, 1)
assert block.to_list() == [[1], [3]], "Block extraction failed."

# Test row and column extraction
row_0 = A.get_row(0)
assert row_0.to_list() == [[1, 2]], "Row extraction failed."

col_1 = A.get_column(1)
assert col_1.to_list() == [[2], [4]], "Column extraction failed."

# Test matrix creation helper functions
zeros_matrix = create_zero_matrix(3, 3)
assert zeros_matrix.to_list() == [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], "Zero matrix creation failed."

ones_matrix = create_one_matrix(2, 2)
assert ones_matrix.to_list() == [[1.0, 1.0], [1.0, 1.0]], "Ones matrix creation failed."

constant_matrix = create_constant_matrix(2, 3, 7)
assert constant_matrix.to_list() == [[7, 7, 7], [7, 7, 7]], "Constant matrix creation failed."

# Test matrix equality
matrix_1 = Matrix([[1, 2], [3, 4]])
matrix_2 = Matrix([[1, 2], [3, 4]])
assert matrix_1 == matrix_2, "Matrix equality check failed."

# Test scalar multiplication
scaled_matrix = A * 2
assert scaled_matrix.to_list() == [[2, 4], [6, 8]], "Scalar multiplication failed."

# Output matrices
print("Matrix A:")
print(A)
print("Matrix B:")
print(B)
print("Matrix C:")
print(C)
print("Matrix I (Identity):")
print(I)

print("Matrix operations worked successfully!")


Matrix A:
1	2
3	4
Matrix B:
5	6
7	8
Matrix C:
9	10
11	12
Matrix I (Identity):
1	0
0	1
Matrix operations worked successfully!
